## Notes
I am assuming you have a train/test/eval split in train_images_5_class so I can just get the images for testing from each category from there
ie
<pre>
Root
├──Input
│  └──train_images_5_class
│     ├──train
│     │  └──..
│     ├──eval
│     │  └──..
│     └──test
│        ├──alert
│        ├──angry
│        ├──frown
│        ├──happy
│        └──relax
└──notebooks
   └──Evaluation.ipynb
</pre>

In [13]:
# import kagglehub
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import random
import torch.nn as nn
from torchvision.models import efficientnet_b5, EfficientNet_B5_Weights
from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url
import torch.optim as optim
import torch
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [14]:
path="../input/actual/test"

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

allTestSet = datasets.ImageFolder(root=path, transform=transform)

print(f"Test set: {len(allTestSet)}")
dataloader = DataLoader(allTestSet, batch_size=32, shuffle=True)


Test set: 1405


In [15]:
# Number of classes in your dataset
num_classes = len(dataloader.dataset.classes) 
print(num_classes)

5


### !Base Model with only final layer retrain

In [ ]:
from torchvision.models import efficientnet_b5
import torch.nn as nn

# Rebuild model structure
model = efficientnet_b5(weights=None)
model.classifier = nn.Sequential(
    nn.Dropout(p=0.4),
    nn.Linear(model.classifier[1].in_features, num_classes)
)

# Load weights
model.load_state_dict(torch.load("../models/efficientnet_b5_finetuned.pth"))

RuntimeError: Error(s) in loading state_dict for EfficientNet:
	Missing key(s) in state_dict: "classifier.1.weight", "classifier.1.bias". 
	Unexpected key(s) in state_dict: "classifier.4.weight", "classifier.4.bias", "classifier.1.interaction.0.weight", "classifier.1.interaction.2.weight". 

In [21]:
import torch

# Move model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

target_names = allTestSet.classes
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=target_names))

print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


Test Accuracy: 0.5331
Classification Report:
              precision    recall  f1-score   support

       alert       0.45      0.40      0.42       281
       angry       0.44      0.29      0.35       281
       frown       0.49      0.52      0.50       281
       happy       0.62      0.78      0.69       281
       relax       0.60      0.68      0.64       281

    accuracy                           0.53      1405
   macro avg       0.52      0.53      0.52      1405
weighted avg       0.52      0.53      0.52      1405

Confusion Matrix:
[[112  39  53  51  26]
 [ 60  81  43  60  37]
 [ 36  26 145  18  56]
 [ 19  26   8 220   8]
 [ 23  14  45   8 191]]


### SE Version of the model

In [18]:
from torchvision.models import efficientnet_b5
import torch.nn as nn
import torch

# Rebuild model structure with SqueezeExcitationBlock
class SqueezeExcitationBlock(nn.Module):
    def __init__(self, channels, reduction_ratio=16):
        super(SqueezeExcitationBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.interaction = nn.Sequential(
            nn.Linear(channels, channels // reduction_ratio, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction_ratio, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.interaction(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

# Load EfficientNet structure
model = efficientnet_b5(weights=None)

# Match the classifier structure from training
channels = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Unflatten(1, (channels, 1, 1)),           # restore shape to [B, C, 1, 1]
    SqueezeExcitationBlock(channels),
    nn.Flatten(),                                # back to [B, C]
    nn.Dropout(p=0.4),
    nn.Linear(channels, num_classes)
)

# Load weights
model.load_state_dict(torch.load("../models/SEClassifier.pth"))
model.eval()


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (1): BatchNorm2d(48, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormAct

In [19]:
# Move model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

target_names = allTestSet.classes
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=target_names))

print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


Test Accuracy: 0.5544
Classification Report:
              precision    recall  f1-score   support

       alert       0.44      0.42      0.43       281
       angry       0.46      0.35      0.40       281
       frown       0.50      0.51      0.51       281
       happy       0.70      0.76      0.73       281
       relax       0.62      0.73      0.67       281

    accuracy                           0.55      1405
   macro avg       0.54      0.55      0.55      1405
weighted avg       0.54      0.55      0.55      1405

Confusion Matrix:
[[117  50  53  32  29]
 [ 67  99  42  42  31]
 [ 47  21 144  12  57]
 [ 18  32  11 214   6]
 [ 18  15  39   4 205]]


### SEModified

In [20]:
def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)

class SqueezeExcitationBlock(nn.Module):
    def __init__(self, channels, reduction_ratio=16):
        super(SqueezeExcitationBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.interaction = nn.Sequential(
            nn.Linear(channels, channels // reduction_ratio, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction_ratio, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.interaction(y).view(b, c, 1, 1)
        return x * y.expand_as(x)
class CustomENB5(nn.Module):
    def __init__(self,num_classes):
        super(CustomENB5, self).__init__()
        WeightsEnum.get_state_dict = get_state_dict

        self.base = efficientnet_b5(weights="DEFAULT")
        for param in self.base.parameters():
            param.requires_grad = False
        
        self.seAfterFeature = SqueezeExcitationBlock(
            channels=2048
        )

        self.base.classifier = nn.Sequential(
            nn.Unflatten(1, (2048, 1, 1)),          # Reshape [B, C] → [B, C, 1, 1]
            SqueezeExcitationBlock(2048),
            nn.Flatten(),                               # Back to [B, C]
            nn.Dropout(p=0.4),
            nn.Linear(2048, num_classes)
        )

    def forward(self, x):
        x = self.base.features(x)
        x = self.seAfterFeature(x)
        x = self.base.avgpool(x)
        x = torch.flatten(x,1)
        x = self.base.classifier(x)
        return x        

In [22]:
model = CustomENB5(num_classes)
model.load_state_dict(torch.load("../models/bestSEModified.pth"))
model.eval()

CustomENB5(
  (base): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(48, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
              (1): BatchNorm2d(48, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scale_a

In [23]:
# Move model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

target_names = allTestSet.classes
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=target_names))

print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


Test Accuracy: 0.5559
Classification Report:
              precision    recall  f1-score   support

       alert       0.47      0.44      0.45       281
       angry       0.50      0.41      0.45       281
       frown       0.49      0.47      0.48       281
       happy       0.70      0.73      0.72       281
       relax       0.59      0.73      0.65       281

    accuracy                           0.56      1405
   macro avg       0.55      0.56      0.55      1405
weighted avg       0.55      0.56      0.55      1405

Confusion Matrix:
[[123  49  48  32  29]
 [ 50 115  36  43  37]
 [ 50  24 133   8  66]
 [ 23  32  11 206   9]
 [ 18  12  41   6 204]]
